---
title: "Introducción a la Programación en Python"
subtitle: "Clase 7 — Input/Output (I/O) y Pathlib"
author: "Dr. Stefan Vogt Geisse"
format:
  revealjs:
    theme: white
    slide-number: true
    transition: fade
    incremental: true
    code-line-numbers: true
    controls: true
    progress: true
    center: true
execute:
  echo: true
  warning: false
  message: false
  error: true
jupyter: python3
---



<!-- 00:00–00:20 (0:20) -->

# Clase 7 — IO y filesystem con `pathlib`
<!-- 00:20–02:30 (2:10) -->

::: notes
- Objetivo de la clase: construir un modelo mental correcto de archivos, carpetas y rutas; leer/escribir texto; usar `pathlib`.
- Recordatorio: hoy NO es análisis de datos; eso viene después.
:::

## Objetivos
<!-- 02:30–05:00 (2:30) -->

1. Diferenciar **memoria** vs **disco** (por qué existen archivos)
2. Leer y escribir **archivos de texto** con `open()` + `with`
3. Entender modos: `"r"`, `"w"`, `"a"` (leer, sobrescribir, anexar)
4. Modelar el filesystem: **archivos vs directorios**, **rutas relativas vs absolutas**
5. Usar `pathlib.Path` para trabajar con rutas de forma portable

## Motivación: por qué los programas necesitan archivos
<!-- 05:00–08:30 (3:30) -->

::: {.columns}
::: {.column width="55%"}
- Un programa vive en **memoria (RAM)** mientras se ejecuta  
- Al terminar, la memoria se libera  
- Si queremos **persistencia**: guardamos en **disco**  
- Archivos permiten:
  - guardar resultados
  - registrar procesos (logs)
  - compartir datos con otros programas
:::

::: {.column width="45%" .fragment}
**Consideraciones Importantes**

- RAM: rápido, temporal  
- Disco: más lento, permanente  

`programa → (RAM) → archivo.txt (disco)`
:::
:::

::: notes
- Enfatizar: IO es el “puente” entre programa y sistema operativo.
- Pregunta rápida a estudiantes: ¿qué cosas ya usan archivos a diario?
:::

## ¿Qué es IO en Python?
<!-- 08:30–11:00 (2:30) -->

- **Input**: leer datos desde afuera del programa  
- **Output**: escribir datos hacia afuera del programa  
- En esta clase: **texto plano**
  - un archivo de texto es una **secuencia de caracteres**
  - cuando lo leemos, trabajamos con **strings**

## Filesystem: archivos vs directorios
<!-- 11:00–13:30 (2:30) -->

- **Directorio (carpeta)**: contiene archivos y otros directorios  
- **Archivo**: contiene datos (texto, imágenes, etc.)  
- Una **ruta (path)** localiza algo en el filesystem

::: {.fragment}
Ejemplos:

- `./report.txt`  (archivo en el directorio actual)
- `./data/`       (directorio llamado `data`)
:::

In [ ]:
# Setup (ejecuta esta celda una vez)
from pathlib import Path

print("Python IO + pathlib — listo.")
print("Directorio de trabajo:", Path.cwd())

## Abrir archivos: IO “clásico”
<!-- 13:30–18:00 (4:30) -->

::: {.columns}
::: {.column width="55%"}
Función central:

```python
open(ruta, modo)
```

Modos comunes:

- `"r"`: leer (falla si no existe)
- `"w"`: escribir (sobrescribe)
- `"a"`: anexar al final (append)

Regla práctica: **usar `with`**.
:::

::: {.column width="45%" .fragment}
Patrón recomendado:

```python
with open("archivo.txt", "r") as f:
    texto = f.read()
```

`with` asegura que el archivo se **cierra** correctamente.
:::
:::

::: notes
- Mostrar la idea de “recurso” (archivo abierto) y por qué `with` lo gestiona.
:::

In [ ]:
# Demo: crear una carpeta de trabajo para esta clase
work = Path.cwd() / "clase7_io"
work.mkdir(exist_ok=True)

print("Carpeta de trabajo:", work)

## Escribir (sobrescribir) vs agregar (append)
<!-- 18:00–21:00 (3:00) -->

- `"w"` **sobrescribe** el archivo completo
- `"a"` **agrega** al final, manteniendo el contenido previo


In [ ]:
# Demo: "w" (write) vs "a" (append)
p = work / "demo_log.txt"

# 1) escribir desde cero
with open(p, "w", encoding="utf-8") as f:
    f.write("Inicio del log\n")

# 2) anexar líneas
with open(p, "a", encoding="utf-8") as f:
    f.write("Paso 1: ok\n")
    f.write("Paso 2: ok\n")

print("Escribimos:", p)
print(p.read_text(encoding="utf-8"))

## Leer archivos de texto
<!-- 21:00–25:00 (4:00) -->

Tres patrones típicos:

1. `f.read()`  
   - lee todo a un string (útil si el archivo es pequeño)
2. `f.readline()`  
   - lee una línea (incluye `\n` al final si existe)
3. Iterar: `for line in f:`  
   - procesa línea por línea (escalable)

In [ ]:
# Crear un archivo de ejemplo (dentro del notebook)
sample = work / "mediciones.txt"
contenido = (
    "2026-01-21 09:00 temp=21.4 estado=OK\n"
    "2026-01-21 09:05 temp=21.6 estado=OK\n"
    "2026-01-21 09:10 temp=22.1 estado=OK\n"
    "2026-01-21 09:15 temp=23.8 estado=WARN\n"
)
sample.write_text(contenido, encoding="utf-8")

print("Archivo creado:", sample)

## Demo: `read()`
<!-- 25:00–28:00 (3:00) -->

In [ ]:
with open(sample, "r", encoding="utf-8") as f:
    txt = f.read()

print(txt)

## Demo: `readline()`
<!-- 28:00–30:00 (2:00) -->

In [ ]:
with open(sample, "r", encoding="utf-8") as f:
    first = f.readline()
    second = f.readline()

print("Primera:", first, end="")
print("Segunda:", second, end="")

## Demo: iterar línea por línea
<!-- 30:00–34:00 (4:00) -->

Objetivo: contar cuántas líneas tienen `estado=WARN`.

In [ ]:
warn = 0

with open(sample, "r", encoding="utf-8") as f:
    for line in f:
        if "estado=WARN" in line:
            warn += 1

print("WARN:", warn)

## Ejercicio 1: leer y extraer una columna
<!-- 34:00–41:00 (7:00) -->

Tienes el archivo `mediciones.txt` dentro de `clase7_io/`.

Tarea:

1. Léelo línea por línea
2. Para cada línea, imprime solo el valor de `temp=...`
3. Mantén el valor como string (por ahora)

Pista: puedes usar `split()`.

::: notes
- Dar 4–5 minutos, luego mostrar solución.
- Si se atascan: sugerir `parts = line.split()` y luego buscar el ítem que empieza con `"temp="`.
:::

## Solución

## Ejercicio 1 — solución
<!-- 41:00–44:00 (3:00) -->

In [ ]:
with open(sample, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.split()
        for item in parts:
            if item.startswith("temp="):
                print(item)   # ejemplo: temp=21.4

## Escribir archivos de texto
<!-- 44:00–48:00 (4:00) -->

Caso típico: escribir un **reporte** o un **log**.

- `f.write("texto\n")` escribe strings
- Conviene agregar `\n` para separar líneas

Patrón:

```python
with open("salida.txt", "w") as f:
    f.write("línea 1\n")
    f.write("línea 2\n")
```

## Ejercicio 2: crear un log de resultados
<!-- 48:00–55:00 (7:00) -->

Tarea:

1. Lee `mediciones.txt`
2. Crea un archivo `solo_warn.txt`
3. Escribe allí **solo** las líneas que contienen `estado=WARN`

Requisitos:

- usa `with`
- usa modo `"w"`
- guarda el archivo dentro de `clase7_io/`

::: notes
- Dar 5 minutos; luego solución.
- Recalcar: el filtro es un `if` dentro del `for`.
:::

In [ ]:
# Tu código aquí (Ejercicio 2)

## Ejercicio 2 — solución
<!-- 55:00–57:00 (2:00) -->

In [ ]:
solo_warn = work / "solo_warn.txt"

with open(sample, "r", encoding="utf-8") as fin:
    with open(solo_warn, "w", encoding="utf-8") as fout:
        for line in fin:
            if "estado=WARN" in line:
                fout.write(line)

print("Escribimos:", solo_warn)
print(solo_warn.read_text(encoding="utf-8"))

## Por qué las rutas importan
<!-- 57:00–61:00 (4:00) -->

Problemas típicos cuando usamos strings “a mano”:

- rutas que dependen del sistema operativo (`/` vs `\`)
- rutas relativas que cambian si cambias el directorio actual
- archivos que no existen (y el programa falla)

Necesitamos una forma **explícita y portable** de manejar paths.

## Rutas relativas vs absolutas
<!-- 61:00–64:00 (3:00) -->

- **Absoluta**: desde la raíz del sistema (única)
  - ejemplo: `/home/usuario/proyecto/clase7_io/mediciones.txt`
- **Relativa**: desde el directorio actual
  - ejemplo: `clase7_io/mediciones.txt`

En un notebook: el “directorio actual” es donde estás ejecutando Jupyter.

In [ ]:
# Demo: ver Path.cwd() y construir rutas relativas
print("cwd:", Path.cwd())
print("ruta relativa:", work / "mediciones.txt")

## Introducción a `pathlib`
<!-- 64:00–69:00 (5:00) -->

`pathlib` (standard library) te da un objeto **Path** en vez de strings.

```python
from pathlib import Path
p = Path("clase7_io") / "mediciones.txt"
```

Ventajas:

- composición de rutas con `/`
- métodos útiles:
  - `.exists()`
  - `.is_file()`, `.is_dir()`
  - `.iterdir()`

In [ ]:
# Demo: inspección con pathlib
p_file = work / "mediciones.txt"
p_dir = work

print("exists:", p_file.exists())
print("is_file:", p_file.is_file())
print("is_dir:", p_dir.is_dir())

## Demo: listar contenidos de un directorio con `.iterdir()`
<!-- 69:00–72:00 (3:00) -->

In [ ]:
for item in work.iterdir():
    print(item.name)

## IO + `pathlib` juntos
<!-- 72:00–75:00 (3:00) -->

Puedes combinar `Path` con `open()`:

```python
p = Path("clase7_io") / "mediciones.txt"
with open(p, "r", encoding="utf-8") as f:
    ...
```

También puedes usar atajos útiles:

- `p.read_text(encoding="utf-8")`
- `p.write_text(texto, encoding="utf-8")`

In [ ]:
# Demo: leer con Path.read_text()
txt = sample.read_text(encoding="utf-8")
print(txt.splitlines()[0])

In [ ]:
# Demo: escribir con Path.write_text()
out = work / "resumen_rapido.txt"
out.write_text("Resumen rápido: archivo creado desde pathlib.\n", encoding="utf-8")

print("OK:", out.exists(), out)

## Final boss
<!-- 75:00–87:00 (12:00) -->

Vas a construir un mini “pipeline” de texto:

1. Leer un archivo de texto
2. Hacer procesamiento mínimo (solo `for` + `if`)
3. Escribir un resumen a disco

Archivo de entrada: `mediciones.txt`

Reglas:

- Cuenta:
  - `n_total`: número total de líneas
  - `n_warn`: cuántas contienen `estado=WARN`
- Extra:
  - encuentra la **temperatura máxima** observada (`temp=...`)
- Escribe un archivo `summary.txt` con 3 líneas:
  - `total=<...>`
  - `warn=<...>`
  - `max_temp=<...>`

::: notes
- Dar 8–9 minutos.
- Recordar: para extraer el número, pueden hacer `item.split("=")[1]` y convertir a `float`.
- Normalizar errores: archivo no encontrado, cast a float, etc.
::

In [ ]:
# Tu código aquí (Final boss)

## Final boss — solución
<!-- 87:00–94:00 (7:00) -->

In [ ]:
n_total = 0
n_warn = 0
max_temp = None

with open(sample, "r", encoding="utf-8") as f:
    for line in f:
        n_total += 1
        if "estado=WARN" in line:
            n_warn += 1

        # buscar temp=...
        for item in line.split():
            if item.startswith("temp="):
                t = float(item.split("=")[1])
                if (max_temp is None) or (t > max_temp):
                    max_temp = t

summary = work / "summary.txt"
with open(summary, "w", encoding="utf-8") as out:
    out.write(f"total={n_total}\n")
    out.write(f"warn={n_warn}\n")
    out.write(f"max_temp={max_temp}\n")

print("Escribimos:", summary)
print(summary.read_text(encoding="utf-8"))

## Cierre
<!-- 94:00–98:00 (4:00) -->

Ahora deberías poder:

- explicar memoria vs disco (por qué existen archivos)
- leer y escribir texto con `open()` + `with`
- elegir entre `"w"` (sobrescribir) y `"a"` (append)
- razonar sobre rutas relativas vs absolutas
- usar `pathlib.Path` para construir rutas y verificar archivos

::: notes
- Conectar con la próxima clase: “procesamiento de datos” (y más adelante pandas) necesita estos fundamentos.
:::